<a href="https://colab.research.google.com/github/Jingut/EX/blob/main/Exp_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 읽어오기

In [1]:
import tensorflow as tf
import glob
import os
from sklearn.model_selection import train_test_split
import re
import numpy as np

txt_file_path = '/content/drive/MyDrive/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)        

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 227088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [2]:
len(raw_corpus)

227088

# 데이터 정제

In [3]:
for idx, sentence in enumerate(raw_corpus):
  if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
  if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
      
  print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


In [4]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip() 
  sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
  sentence = re.sub(r'[" "]+', " ", sentence) 
  sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
  sentence = sentence.strip() 
  sentence = '<start> ' + sentence + ' <end>' 
  return sentence

In [5]:
corpus = []

for sentence in raw_corpus:
  # 우리가 원하지 않는 문장은 건너뜁니다
  if len(sentence) == 0: continue
  if sentence[-1] == ":": continue
  # 정제를 하고 담아주세요
  preprocessed_sentence = preprocess_sentence(sentence)
  corpus.append(preprocessed_sentence)

In [6]:
def tokenize(corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
  num_words=12000, 
  filters=' ',
  oov_token="<unk>")
  # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
  tokenizer.fit_on_texts(corpus)
  # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
  tensor = tokenizer.texts_to_sequences(corpus)   
  # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
  # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
  # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=21)#21로 길이제한
  
  print(tensor,tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  339   26 ...    0    0    0]
 [   2  202   13 ...    0    0    0]
 [   2   25   16 ...    0    0    0]
 ...
 [   2  848    1 ...    0    0    0]
 [   2  161   64 ...    0    0    0]
 [   2 4396  161 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f88c478f650>


In [7]:
len(tensor)

199764

In [8]:
len(tensor[5])

21

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])
# 패딩을 앞에둘지 뒤에둘지에 따라 성능이 달라짐

[   2  339   26  100 5027    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[ 339   26  100 5027    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


# 평가 데이터셋 분리

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, random_state=20, shuffle = True, test_size=0.2)

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
print(dataset)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(20,), dtype=tf.int32, name=None), TensorSpec(shape=(20,), dtype=tf.int32, name=None))>


<BatchDataset element_spec=(TensorSpec(shape=(256, 20), dtype=tf.int32, name=None), TensorSpec(shape=(256, 20), dtype=tf.int32, name=None))>

In [11]:
len(src_input[0])

20

# 인공지능 만들기


In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [13]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
      super().__init__()
      
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
      self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
      self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
      self.linear = tf.keras.layers.Dense(vocab_size)
      
  def call(self, x):
      out = self.embedding(x)
      out = self.rnn_1(out)
      out = self.rnn_2(out)
      out = self.linear(out)
      
      return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 12001), dtype=float32, numpy=
array([[[ 5.61067827e-05, -2.35063751e-04,  1.03112237e-04, ...,
          9.20737293e-05, -9.88590691e-05,  9.26610446e-05],
        [ 5.06080396e-05, -2.82036082e-04,  4.02501100e-05, ...,
          1.83709810e-04, -1.45609665e-04, -3.44066848e-06],
        [ 2.85499700e-04, -3.46019689e-04,  6.62462844e-05, ...,
          3.33856908e-04, -2.27887373e-04, -2.15124703e-04],
        ...,
        [ 3.42460210e-03,  4.60605137e-04, -2.48633535e-03, ...,
         -9.51610506e-04,  6.26751687e-04,  1.89533865e-03],
        [ 3.86591791e-03,  6.18351041e-04, -2.83506117e-03, ...,
         -1.04904699e-03,  6.65441388e-04,  2.11543194e-03],
        [ 4.27390961e-03,  7.42815610e-04, -3.10622714e-03, ...,
         -1.13556674e-03,  7.11450179e-04,  2.28946772e-03]],

       [[ 5.61067827e-05, -2.35063751e-04,  1.03112237e-04, ...,
          9.20737293e-05, -9.88590691e-05,  9.26610446e-05],
        [ 8.43302332e-05, -3.03600100e-04,  1

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [16]:
# optimizer와 loss등은 차차 배웁니다 
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

# model.fit() 함수에 들어가는 다양한 인자를 알고 싶다면 아래의 문서를 참고하세요. 
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit


LSTM = model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
4995/4995 [==============================] - 162s 32ms/step - loss: 2.4694 - val_loss: 2.2791
Epoch 2/10
4995/4995 [==============================] - 158s 32ms/step - loss: 2.1660 - val_loss: 2.1471
Epoch 3/10
4995/4995 [==============================] - 159s 32ms/step - loss: 1.9964 - val_loss: 2.0741
Epoch 4/10
4995/4995 [==============================] - 158s 32ms/step - loss: 1.8489 - val_loss: 2.0305
Epoch 5/10
4995/4995 [==============================] - 158s 32ms/step - loss: 1.7133 - val_loss: 2.0113
Epoch 6/10
4995/4995 [==============================] - 159s 32ms/step - loss: 1.5887 - val_loss: 2.0019
Epoch 7/10
4995/4995 [==============================] - 158s 32ms/step - loss: 1.4760 - val_loss: 2.0009
Epoch 8/10
4995/4995 [==============================] - 159s 32ms/step - loss: 1.3748 - val_loss: 2.0093
Epoch 9/10
4995/4995 [==============================] - 158s 32ms/step - loss: 1.2845 - val_loss: 2.0298
Epoch 10/10
4995/4995 [==============================] 

# 인공지능 평가


In [17]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [18]:
'''LSTM = model.fit(enc_train, dec_train, epochs=10, shuffle = True, validation_data=(enc_val, dec_val))

# 진행도 확인
import matplotlib.pyplot as plt
plt.plot(LSTM .history['loss'])
plt.plot(LSTM .history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()'''

"LSTM = model.fit(enc_train, dec_train, epochs=10, shuffle = True, validation_data=(enc_val, dec_val))\n\n# 진행도 확인\nimport matplotlib.pyplot as plt\nplt.plot(LSTM .history['loss'])\nplt.plot(LSTM .history['val_loss'])\nplt.ylabel('Loss')\nplt.xlabel('Epoch')\nplt.legend(['train_loss', 'val_loss'])\nplt.show()"

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much , i love you so much , i love '

#  결론
이번 프로젝트에서는 RNN으로 여러 가사들을 학습시켜 인공지능 작사를 하는 모델을 학습시켜 보았습니다.


데이터 전처리 과정이 매우 까다로웠는데 정규표현식을 이용하여 학습에 필요없는 특수문자나 공백을 제거하고 문장 토큰화를 거쳐 RNN모델에 통과시켜 특수한 단어나 문장을 지정해주었을 때 학습된 모델이 어떻게 작사를 해내는지까지 구현해보았습니다.




# 회고

---



## - 이번 프로젝트에서 어려웠던 점

* 기본적인 전처리는 어느정도 되어 있어 간단하였지만 루브릭지표를 맞추는데 꽤나 시간이 걸렸고 학습시간 또한 만만치 않아서 애를 먹었습니다.

> 그리고 추천해주신 토큰의 길이가 15였는데 직접 그렇게 설정해보니 하이퍼파라미터를 어떻게 설정하여도 val_loss가 2.4 밑으로 내려가지 않아 힘들었는데 max_len의 길이를 조금 더 늘였더니 val_loss가 눈에 띄게 떨어져서 해결이 되는 부분이 있었습니다.

> 또한 주어진 조건인 10 에포크 제한이 있어 힘들었던 것 같습니다. 조금만 더 돌렸다면 val_loss가 2.2 이하로 떨어졌을 수 있지 않을까라는 생각도 해보았습니다.





---



## - 루브릭 평가지표를 맞추기 위해 시도한 것들

* max_len을 15로 지정하였을 때 제가 시도해본 것들은 일단 rnn의 레이어 수를 최대 6층까지 
확장 해보았었습니다.


* 모델의 Embedding Size와 Hidden Size를 2배수 단위로 늘여서 진행해보았습니다.


* 학습시간을 줄여보려 배치사이즈 또한 2배수로 늘려가면서 진행해보았습니다.


* padding을 pre, post 로 나눠 진행해보았습니다.


* 그리고 주어진 파일에 동일인의 파일인데 -,_ 차이로 파일의 문장순서만 다른 것이 있어서 중복 텍스트 파일을 삭제하고도 진행해보았습니다.

---


## - 프로젝트를 진행하면서 알게된 점

* max_len을 어떻게 설정해주느냐에 따라 모델이 이해하는 문맥의 수준이 올라간다는 것을 val_loss를 통해 알수 있었습니다.


* 모델의 Embedding Size와 Hidden Size를 늘려 보았지만 학습시간도 늘고 val_loss도 늘었습니다...그냥 무작정 늘리기만하면 능사가 아니라는 것을 깨달았습니다.


* 이번 프로젝트에서 배치사이즈를 늘린다고 학습시간이 눈에 띄게 줄어들지 않는다는 것을 알았습니다.


* 제가 생각하기엔 RNN에서는 뒤편에 padding을 넣는 것 보다는 앞쪽에 padding을 넣어주는 편이 마지막 결과에 paddind이 미치는 영향이 적어져서 더 좋은 성능을 낼 것이라고 생각했었는데 
실제로 돌려보니 post로 지정해주는 것이 로스값이 적게 나와서 패딩을 post로 두고 진행하였습니다.  


* 텍스트파일의 중복을 제거해주면 조금 더 나아질 것 같아 진행해 보았지만 val_loss 값의 별 차이는 없었습니다.



---


## - 자기다짐

* cv를 열심히 해야겠다고 생각했습니다.

* 자연어처리 전처리가 쉽지 않다는 것을 느꼈고 정규표현식, 패딩에 이해와 관련된 공부가 필요하다고 생각했습니다.
 

